# Lecture 8: MQL 1

Gittu George, Feb 01, 2024

_Attribution: This notebook is developed using materials from DSCI 513 by Arman._

## Announcements
- 2 more weeks before the final exam.
    - No worksheet 10, During that time we will have open OHs - clarify questions before the final exam.
- Piazza Communication on Exam Days: 
    - To encourage appropriate study habits, we will stop holding office hours / answering questions on the discussion 1 day before an exam (ie there won't be office hours/answering questions on Tuesday Feb 13, but piazza board is open if students want to help each other ).
- Exam preparation ***Special Office Hours***
    - Kate: 9 -10 AM
    - Daniel: 12 - 1 PM
    - Ngoc: 4 - 5 PM
    - Jordan: 9 - 10 PM
    - Zoom link: [https://ubc.zoom.us/j/67444402927?pwd=RXEyN1RNUGh0cXVUMVRaM2NEaVVLdz09](https://ubc.zoom.us/j/67444402927?pwd=RXEyN1RNUGh0cXVUMVRaM2NEaVVLdz09)
- Be careful with the use of tools like ChatGPT.

## Agenda
- Introduction to NoSQL document database (MongoDB)
- Basic queries in MongoDB
- Class activity time.

## Objectives

- Setting up mongoDB instance in cloud
- Write basic MQL

### MongoDB interfaces

#### MongoDB shell (`mongosh`)

This is command line interface for interacting with a MongoDB database, similar to `psql` that we've used for Postgres. `mongosh` is based on the JavaScript language. We will not use `mongosh` much in this course.

#### MongoDB Compass
Compass is a versatile graphical user interface for using MongoDB databases. This is a similar application to pgAdmin that we've used for Postgres.

#### MongoDB's Python driver (`pymongo`)
And finally, `pymongo` is the official Python driver for MongoDB. If your using the course `conda` environment, this package is installed and ready to use in Jupyter Lab. You can take a look at `pymongo`s documentation [here](https://pymongo.readthedocs.io/en/stable/tutorial.html).

In [28]:
from pymongo import MongoClient
import json
import urllib.parse

with open('credentials_mongodb.json') as f:
    login = json.load(f)

username = login['username']
password = urllib.parse.quote(login['password'])
host = login['host']
url = "mongodb+srv://{}:{}@{}/?retryWrites=true&w=majority".format(username, password, host)

In [29]:
client = MongoClient(url)

In [30]:
client.list_database_names()

['mds',
 'sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_guides',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'admin',
 'local']

## MongoDB query language (MQL)

<img src="img/nosql.png" width="400">

([image source](https://dataedo.com/cartoon/it-is-nosql))

```{admonition} See also ...
SQL to MongoDB mapping chart: https://docs.mongodb.com/manual/reference/sql-comparison/
```

As mentioned earlier, there is no standard query language among NoSQL DBMSs. This is because each NoSQL DBMS supports a different data model and obviously no one language can suit all data models.

MongoDB has its own query language known as MongoDB Query Language or MQL (We already saw CQL for neo4j). I will walk you through the usage of MQL in the remainder of this lecture.

### Accessing databases and collections

Here is how we can access databases through different interfaces.


**`Compass`**:

It's just point and click. I'll demo this in class.

```{figure} img/explore.png
---
width: 200px
align: center
---
```

```{margin}
<img src='img/demos.png'>
```

**`mongosh`**:
```js
show dbs
use my_db
```

**`pymongo`**:

```
my_db = client['my_db']
my_db
```

Running the above cell just gives you some information about our connection to the server. We'll learn how to run queries on this connection in a bit. For now, let's see what databases we have:

In [31]:
client.list_database_names()

['mds',
 'sample_airbnb',
 'sample_analytics',
 'sample_geospatial',
 'sample_guides',
 'sample_mflix',
 'sample_restaurants',
 'sample_supplies',
 'sample_training',
 'sample_weatherdata',
 'admin',
 'local']

To access collections withing each database, use the following syntax:

**`mongosh`**:
```js
db.my_collection.method()
```

**`pymongo`**:

```
my_collection = my_db['my_collection']
my_collection
```

Again, some information that we don't need. We will never use the database or collection objects simple like this. For now, let's take a look at the collections inside the `sample_mflix` database:

In [32]:
client['sample_mflix'].list_collection_names()

['sessions', 'theaters', 'comments', 'users', 'movies', 'embedded_movies']

Or alternatively:

In [33]:
client.sample_mflix.list_collection_names()

['sessions', 'theaters', 'comments', 'users', 'movies', 'embedded_movies']

A very important thing to know before using MQL is that

> **Everything in MongoDB is a JSON-like document**

even queries themselves!

### `find`

The main method used for querying documents is the `.find()` method. Here is an example of a query in MongoDB:

**`mongosh`**:

```js
db.movies.find( {title: 'Titanic'} )
```

**`pymongo`**:

In [34]:
client['sample_mflix']['movies'].find( filter={'title': 'Titanic'} )

Using `filter=` is optional in the argument list, but if you remember from Python's Zen advice, "_explicit is better than implicit_".

Well, the above code doesn't do anything because it returns a cursor object which is basically a Python generator. Let's return the first element of this generator:

In [35]:
next(client['sample_mflix']['movies'].find( {'title': 'Titanic'} ))

{'_id': ObjectId('573a139af29313caabcefb1d'),
 'plot': 'The story of the 1912 sinking of the largest luxury liner ever built, the tragedy that befell over two thousand of the rich and famous as well as of the poor and unknown passengers aboard the doomed ship.',
 'genres': ['Action', 'Drama', 'History'],
 'runtime': 173,
 'cast': ['Peter Gallagher',
  'George C. Scott',
  'Catherine Zeta-Jones',
  'Eva Marie Saint'],
 'poster': 'https://m.media-amazon.com/images/M/MV5BYWM0MDE3OWMtMzlhZC00YzMyLThiNjItNzFhNGVhYzQ1YWM5XkEyXkFqcGdeQXVyMTczNjQwOTY@._V1_SY1000_SX677_AL_.jpg',
 'title': 'Titanic',
 'fullplot': "The plot focuses on the romances of two couples upon the doomed ship's maiden voyage. Isabella Paradine (Catherine Zeta-Jones) is a wealthy woman mourning the loss of her aunt, who reignites a romance with former flame Wynn Park (Peter Gallagher). Meanwhile, a charming ne'er-do-well named Jamie Perse (Mike Doyle) steals a ticket for the ship, and falls for a sweet innocent Irish girl o

Or we can pass it to `list()` to materialize the generator entirely:

In [36]:
list(
    client['sample_mflix']['movies'].find( {'title': 'Titanic'} )
)

[{'_id': ObjectId('573a139af29313caabcefb1d'),
  'plot': 'The story of the 1912 sinking of the largest luxury liner ever built, the tragedy that befell over two thousand of the rich and famous as well as of the poor and unknown passengers aboard the doomed ship.',
  'genres': ['Action', 'Drama', 'History'],
  'runtime': 173,
  'cast': ['Peter Gallagher',
   'George C. Scott',
   'Catherine Zeta-Jones',
   'Eva Marie Saint'],
  'poster': 'https://m.media-amazon.com/images/M/MV5BYWM0MDE3OWMtMzlhZC00YzMyLThiNjItNzFhNGVhYzQ1YWM5XkEyXkFqcGdeQXVyMTczNjQwOTY@._V1_SY1000_SX677_AL_.jpg',
  'title': 'Titanic',
  'fullplot': "The plot focuses on the romances of two couples upon the doomed ship's maiden voyage. Isabella Paradine (Catherine Zeta-Jones) is a wealthy woman mourning the loss of her aunt, who reignites a romance with former flame Wynn Park (Peter Gallagher). Meanwhile, a charming ne'er-do-well named Jamie Perse (Mike Doyle) steals a ticket for the ship, and falls for a sweet innocent I

In [37]:
# import pandas as pd 
# df = pd.DataFrame(x)
# df

```{note}
`.find( filter={} )` or `.find()` returns every document in the collection.
```

Note that there is another method `.findOne()` in `mongosh` and `.find_one()` in `pymongo`. This method returns only one document regardless of how many there are, according to the order in which documents are stored on the physical disk. It can be 

### `projection`

Remember what projection meant in SQL? Returning a particular set of columns among all that exist in a table was called projection (of the results onto particular columns).

Projection has a similar meaning in NoSQL: it means explicitly choosing the fields that we are interested in, instead of all fields that are returned by default. This is done by feeding a list of fields to the `projection=` argument, as well as a truthy of falsy value that indicates whether or not that field should be included.

For example, here I return the `title` and `year` fields only from the document in the result:

**`mongosh`**:
```js
db.movies.find( {title: 'Titanic'}, {'title': 1, 'year': 1} )
```

**`pymongo`**:

In [38]:
list(
    client['sample_mflix']['movies'].find(
        filter={'title': 'Titanic'},
        projection={'title': 1, 'year': 1}
    )
)

[{'_id': ObjectId('573a139af29313caabcefb1d'),
  'title': 'Titanic',
  'year': 1996},
 {'_id': ObjectId('573a139af29313caabcf0d74'),
  'year': 1997,
  'title': 'Titanic'}]

```{note}
- In `pymongo`, you can use `True` instead of `1` and `False` instead of `0`.

- In `pymongo`, we need to enclose all field names in single or double quotes (e.g. `'title'` not `title`), otherwise Python would complain because it doesn't recognize those names. In `mongosh`, this is not necessary.
```

In the above returned documents, note that the primary key field, namely, the `_id` field is always returned by default unless you explicitly exclude it using `{'_id': 0}` or `{'_id': False}`. **This is the only scenario where we might mix up `1`s and `0`s (or `True`s and `False`s) in the projection field.**

In [39]:
list(
    client['sample_mflix']['movies'].find(
        filter={'title': 'Titanic'},
        projection={'_id': 0, 'title': 1, 'year': 1}
    )
)

[{'title': 'Titanic', 'year': 1996}, {'year': 1997, 'title': 'Titanic'}]

### `sort`

**`mongosh`**:
```js
db.movies.find(<filter>, <projection>).sort( {runtime: 1, year:-1} )
```

**`pymongo`**:

In [40]:
list(
    client['sample_mflix']['movies'].find(
        filter={'title': 'Titanic'},
        projection={'_id': 0, 'title': 1, 'year': 1, 'runtime': 1},
        sort=[('runtime', 1), ('year', -1)]
    )
)

[{'runtime': 173, 'title': 'Titanic', 'year': 1996},
 {'year': 1997, 'title': 'Titanic', 'runtime': 194}]

### `limit`

**`mongosh`**:
```js
db.movies.find({}, {title: 1, _id: 0}).limit(5)
```

**`pymongo`**:

In [41]:
list(
    client['sample_mflix']['movies'].find(
        projection={'title': 1, '_id': 0},
        limit=5
    )
)

[{'title': 'The Great Train Robbery'},
 {'title': 'A Corner in Wheat'},
 {'title': 'Winsor McCay, the Famous Cartoonist of the N.Y. Herald and His Moving Comics'},
 {'title': 'Traffic in Souls'},
 {'title': 'Gertie the Dinosaur'}]

### `count` and `count_documents`

**`mongosh`**:
```js
db.movies.find({year:2000}).count()
db.movies.countDocuments()
```

**`pymongo`**:

In [42]:
client['sample_mflix']['movies'].count_documents(filter={'year': 2000})

581

### `skip`

**`mongosh`**:
```js
db.movies.find( filter={title: 'Titanic'}, projection={'title': 1, 'year': 1} ).skip(2)
```

**`pymongo`**:

In [43]:
list(
    client['sample_mflix']['movies'].find(
        filter={'title': 'Titanic'},
        projection={'title': 1, 'year': 1},
        skip=2
    )
)

[]

### `distinct`

**`mongosh`**:
```js
db.movies.distinct( 'title', {title: 'Titanic'} )
```

**`pymongo`**:

In [44]:
list(
    client['sample_mflix']['movies'].find(
        filter={'title': 'Titanic'},
    ).distinct('title')
)

['Titanic']

The `distinct` method here only returns unique **values**, not entire documents.

## Can you?

- list down the benefits of having a document database 
- Set up MongoDB cloud instance?
- Various ways to interact with MongoDB
- Do basic MQL query

## Class activity

```{note}
Check worksheet 8 in canvas for detailed instructions on activity and submission instructions.
```
- Set up MongoDB in the cloud.
- Practice MQL.